# MIIC

Prepare files to send to the MIIC web platform.

# 0. Loading the libraries

In [7]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt

import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

In [8]:
# Set up constants
save_folder = "figures/"
objects_folder = "saved_objects/"
sc.settings.figdir = './'+save_folder

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3

# 1. MIIC function

In [9]:
with open('saved_objects/adata_annotated.pkl', 'rb') as inp:
    adata = pickle.load(inp)

adata.X = adata.raw.X

In [10]:
def miic(lgenes, data, name):

    gene_data = pd.DataFrame(columns = lgenes)
    for gene in lgenes:
        gene_data[gene] = [e[0] for e in data.X[:, data.var.gene_name == gene].tolist()]
    miic_data = pd.concat([ gene_data.reset_index(drop=True), pd.DataFrame(data.obs[['Il10_positive', 'Il10_high']]).reset_index(drop=True) ], axis=1)

    miic_data.to_csv("miic/"+name+"_data.csv", header = True, index = False)
    miic_data.to_excel("miic/"+name+"_data.xlsx", header = True, index = False)

    supp = pd.DataFrame(columns = ['var_names', 'var_type', 'levels_increasing_order', 'is_contextual', 'group'])
    supp['var_names'] = miic_data.columns
    supp['var_type'] = [1]*len(lgenes) + [0]*(len(miic_data.columns) - len(lgenes))
    supp['levels_increasing_order'] = ['']*len(lgenes) + ['Il10-,Il10+', 'Not,High']
    supp['is_contextual'] = [0]*len(lgenes) + [1]*(len(miic_data.columns) - len(lgenes))
    supp['group'] = 'all'

    supp.to_csv("miic/"+name+"_supp.tsv", sep = '\t', header = True, index = False)

# 2. Prepare MIIC files for different gene lists

### 1. Il10 signature

In [11]:
df = pd.read_csv('gene_lists/'+"Il10_signature"+'.csv')
lgenes = df[df.columns[0]].tolist() + ['Il10']

In [12]:
miic(lgenes, adata.copy(), "Il10sig_all")
miic(lgenes, adata[(adata.obs.cond == "N6")].copy(), "Il10sig_N6")
miic(lgenes, adata[(adata.obs.sample2 == "N6-48h")].copy(), "Il10sig_N6-48h")

### 2. N6 signature

In [13]:
df = pd.read_csv('gene_lists/'+"N6vsDMSO_signature"+'.csv')
lgenes = df[df.columns[0]].tolist()

In [14]:
miic(lgenes, adata.copy(), "N6sig_all")
miic(lgenes, adata[(adata.obs.cond == "N6")].copy(), "N6sig_N6")
miic(lgenes, adata[(adata.obs.sample2 == "N6-48h")].copy(), "N6sig_N6-48h")